In [130]:
from pathlib import Path
TSV_PATH = Path("/Users/joregan/Playing/hsi/annotations/word_annotations/main")
JSON_PATH = Path("/Users/joregan/Playing/hsi/annotations/final_resolved")

In [131]:
import json

with open("/Users/joregan/Playing/hsi/annotations/all_meta.json") as inf:
    data = json.load(inf)

In [132]:
data["12"]

{'recording_id': 'hsi_3_0715_209_008',
 'segment_id': '42',
 'phrase': 'the Rug',
 'utterance': 'Did you move the Rug like in front of the...',
 'object_id': 'Rug',
 'object_name': 'Rug',
 'frame_number': 2756,
 'image_paths': {'color': 'images/color/hsi_3_0715_209_008_42_color.png',
  'depth': 'images/depth/hsi_3_0715_209_008_42_depth.png',
  'overlay': 'images/overlay/hsi_3_0715_209_008_42_overlay.png',
  'mask': 'images/masks/hsi_3_0715_209_008_42_mask.png'},
 'topic': 'Rug',
 'timing': {'phrase_start': 183.735,
  'phrase_end': 184.295,
  'utterance_start': 183.5558059638532,
  'utterance_end': 195.32878031395185},
 'meta_id': 12}

In [135]:
old_json = {}

In [ ]:
def get_topic_context(segments, segment_id, size=None, keep_topic=True):
    rec_id = segments[segment_id]["recording_id"]
    orig_seg_id = segments[segment_id]["segment_id"]
    if not "recording_id" in old_json:
        with open(JSON_PATH / f"{rec_id}.json") as inf:
            old_json["recording_id"] = json.load(inf)
    original = old_json["recording_id"]
    orig_keys = list(original.keys())
    orig_topic = original[orig_seg_id]["high_level"]["current_topic"]

    index = orig_keys.index(orig_seg_id)
    if size is None:
        start = 0
    else:
        start = index - size
    ctx_range = orig_keys[start:index]

    if len(ctx_range) < size:
        if int(orig_seg_id) <= size:
            pass
        else:
            print(f"Warning: size of {size} cannot be satisfied: {ctx_range}")
    
    topics = [original[x]["high_level"]["current_topic"] for x in ctx_range]

    tmp = []
    for p in zip(ctx_range, topics):
        if not keep_topic:
            tmp.append(original[p[0]]["snippet"])
        elif keep_topic and p[1] == orig_topic:
            tmp.append(original[p[0]]["snippet"])
        else:
            tmp.append(None)
    return " ".join([x for x in tmp if x is not None])

In [161]:
get_topic_context(data, "12", 5, False)

"Yeah. Okay. Ah so yeah that's really nice that you cleaned some [loud clicking noise] and you tried. Yeah Yeah ok ok."